In [1]:
from __future__ import print_function

In [2]:
training_data = [
['Yellow', 10, 'Giraffe'],
['Yellow', 10, 'Giraffe'],
['Yellow', 6, 'Tiger'],
['Brown', 2, 'Monkey'],
['Gray', 8, 'Elephant'],
['Black',8,'Elephant'],
]

# training_data = [
#  ['Green', 3 ,'Apple'],
#  ['Yellow', 3, 'Apple'],
#  ['Red', 1, 'Grape'],
#  ['Red', 1, 'Lemon']
# ]


In [3]:
header = ["color", "height", "label"]

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [5]:
unique_vals(training_data, 1)

{2, 6, 8, 10}

In [6]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {} 
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
class_counts(training_data)

{'Giraffe': 2, 'Tiger': 1, 'Monkey': 1, 'Elephant': 2}

In [8]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [9]:
class Question:
    """A Question is used to partition a dataset. This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare the feature value in an example to the feature
    value stored in the question. See the demo below. """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [10]:
def partition(rows, question):
    """Partitions a dataset.
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [11]:
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
print(true_rows)
print(false_rows)

[['Yellow', 10, 'Giraffe'], ['Yellow', 10, 'Giraffe'], ['Yellow', 6, 'Tiger']]
[['Brown', 2, 'Monkey'], ['Gray', 8, 'Elephant'], ['Black', 8, 'Elephant']]


In [12]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise.
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [13]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [14]:
current_uncertainty = gini(training_data)
current_uncertainty

0.7222222222222221

In [15]:
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
info_gain(true_rows, false_rows, current_uncertainty)

0.2777777777777777

In [16]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0 # keep track of the best information gain
    best_question = None # keep train of the feature / value that produced
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    for col in range(n_features): # for each feature
        values = set([row[col] for row in rows]) # unique values in the col
    for val in values: # for each value
        question = Question(col, val)

        true_rows, false_rows = partition(rows, question)


        if len(true_rows) == 0 or len(false_rows) == 0:
            continue

        gain = info_gain(true_rows, false_rows, current_uncertainty)

        if gain >= best_gain:
            best_gain, best_question = gain, question
    return best_gain, best_question

In [17]:
best_gain, best_question = find_best_split(training_data)
best_question

Is height >= 10?

In [18]:
class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [19]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
    question,
    true_branch,
    false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [20]:
def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [21]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    # Print the question at this node
    print(spacing + str(node.question))
    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + " ")

In [22]:
my_tree = build_tree(training_data)

In [23]:
print_tree(my_tree)

Is height >= 10?
--> True:
 Predict {'Giraffe': 2}
--> False:
 Is height >= 8?
 --> True:
  Predict {'Elephant': 2}
 --> False:
  Is height >= 6?
  --> True:
   Predict {'Tiger': 1}
  --> False:
   Predict {'Monkey': 1}


In [24]:
def classify(row, node):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [25]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [26]:
# Evaluate
testing_data = [
['Yellow', 6, 'Tiger'],
['Yellow', 5, 'Tiger'],
['Brown', 2, 'Monkey'],
['Brown', 1, 'Monkey'],
['Yellow', 11, 'Giraffe'],
['Black', 8, 'Elephant'],
['Gray', 7, 'Elephant'],
]

In [27]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
    (row[-1], print_leaf(classify(row, my_tree))))

Actual: Tiger. Predicted: {'Tiger': '100%'}
Actual: Tiger. Predicted: {'Monkey': '100%'}
Actual: Monkey. Predicted: {'Monkey': '100%'}
Actual: Monkey. Predicted: {'Monkey': '100%'}
Actual: Giraffe. Predicted: {'Giraffe': '100%'}
Actual: Elephant. Predicted: {'Elephant': '100%'}
Actual: Elephant. Predicted: {'Tiger': '100%'}
